In [1]:
import os
import pandas as pd
from pandas import DataFrame
import numpy as np
import io

In [2]:
#os.chdir(r'datascience/MLCourse')

In [3]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = pd.DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))


In [4]:
from sklearn.model_selection import train_test_split
X = data['message']
y = data.loc[:,['class']]
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(stop_words='english')
X_train_tf = tf.fit_transform(X_train)
X_test_tf = tf.transform(X_test)

In [6]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_tf,y_train.values.ravel())
pred = nb.predict(X_test_tf)

In [7]:
from sklearn.metrics import confusion_matrix as cm
matrix = cm(y_test,pred)


In [8]:
tn,fp=matrix[0]
fn,tp=matrix[1]

In [9]:
accuracy = (tn+tp)/len(y_test)

In [10]:
accuracy*100

90.16666666666666

In [11]:
precision = tp/(fp+tp)

In [12]:
precision

1.0